In [1]:
from datetime import datetime
import json

In [2]:
import anvil.server

anvil.server.connect("client_KYOM4YFDIE4TMEO3UAOTEPRC-55C7JZ62MGB2UXA6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as CLIENT


In [3]:
from anvil.tables import app_tables

questions = app_tables.questions.search()

In [35]:
from dotenv import load_dotenv
from groq import Groq
import json_repair
import os

load_dotenv()

key_groq = os.environ["GROQ_API_KEY"]
client = Groq(api_key=key_groq)

In [5]:
json_schema = """
  {
    "topic_description": {
      "type": "string",
      "description": "A sentence describing the sub-topic to which the question belongs. That means this sentence should specify in a granular level what specific sub-topic the question belongs to. It should be abstract in a way that other questions could be put in this description too. Use between 5 and 10 words."
    },
    "level": {
      "type": "string",
      "description": "The difficulty level of the question. It should be only one of the following options: 'beginner', 'intermediate', 'advanced'."
    },
    "question": {
      "type": "string",
      "description": "The actual question text. It should be a question of type TRUE or FALSE. It means that the questions should be an assertion that could be answered with TRUE or FALSE."
    },
    "answer_correct": {
      "type": "string",
      "description": "The correct answer to the question. It should be only one of the following options: TRUE or FALSE"
    },
    "explanation": {
      "type": "string",
      "description": "An explanation or solution to the question."
    }
  }
"""

In [20]:
prompt_question_generator = """
    TASK CONTEXT:
    I am studying DATASTRUCTURES AND ALGORITHMS and I need to practice some questions on this topic. 
    
    The objective is to memorize some patterns.
    
    TASK DESCRIPTION:
    I would like you to generate a list of TRUE or FALSE questions.
    Each question should be in the form of a statement that could be either TRUE or FALSE.
    I will provide an example code in the CODE section.
    You should not change the code.
    You should not output the code, because it will be showed to the user automaticaly.
    I want questions that ask what the code do. I some questions you should assert something false, in other something true about the code.
    You should think in what this code could be used in a real case, or in DSA questions.

    
    CODE:
    {code}
    
    TASK REQUIREMENTS:
    Please refrain from creating questions that require mathematical calculations, but you may create questions with mathematical formulas.
    You SHOULD use LATEX to write mathematical formulas and code, but you should use the Katex flavor.
    Also you should put $$ in the beggining of the katex code and $$ at the end of the code. This is necessary because the interpreter needs it.
    
    TASK DETAILS:
    You should create {quantity} questions of level {level}.
   
    
    FORMAT OUTPUT INSTRUCTIONS:
    It should be formatted in list of JSON objects as described below.
    {json_schema}
"""

In [21]:
code1 = """
        x = int(input())
        
        a = 10
        b = 5
        
        bigger = max(a, b)
        minor = min(a, b)
        
        if x % bigger != 0 and x % minor != 0:
            print(-1)
        else:
            print( x // bigger + x % bigger // minor )
    """

In [30]:
parameters = {
    "quantity": "5",
    "level": "medium",
    "code": code1,
}

In [34]:
prompt_question_generator_formatted = prompt_question_generator.format(
    code=parameters["code"],
    quantity=parameters["quantity"],
    level=parameters["level"],
    json_schema=json_schema,
)

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": prompt_question_generator_formatted}],
    temperature=1,
    max_tokens=2420,
    top_p=1,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

completion = response.choices[0].message
questions = json_repair.loads(completion.content)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n    "topic_description": "Finding the largest and smallest values in a given sequence",\n    "level": "medium",\n    "question": "$$x % bigger != 0 and x % minor != 0$$ given in the code always evaluates to TRUE.",\n    "answer_correct": "FALSE",\n    "explanation": "The condition in the code checks if the remainder of x when divided by the larger of a and b, or by the smaller of a and b, is not zero. This means it\'s actually looking for specific values of x that satisfy some conditions, so it\'s unlikely to always evaluate to TRUE."\n  },\n  {\n    "topic_description": "Use of conditional statements in programming",\n    "level": "medium",\n    "question": "The given code will always print a value after execution.",\n    "answer_correct": "TRUE

In [ ]:
if not isinstance(questions, list):
        questions = [questions]

for question in questions:
    print(json.dumps(question, indent=4))
    # add to the database
    # app_tables.questions.add_row(
    #     created_at=datetime.now(),
    #     title=parameters["title"],
    #     topic_description=question["topic_description"],
    #     level=question["level"],
    #     question=question["question"],
    #     type="true_or_false",
    #     answer_correct=question["answer_correct"],
    #     answers=None,
    #     explanation=question["explanation"],
    #     user=anvil.users.get_user(),
    # )

{
    "topic_description": {
        "type": "string",
        "description": "Finding the maximum and minimum numbers in Python"
    },
    "level": "easy",
    "question": {
        "type": "string",
        "text": "Does this code always print the same result for a given input?"
    },
    "answer_correct": {
        "type": "string",
        "text": "FALSE"
    },
    "explanation": {
        "type": "string",
        "text": "The code depends on the input value and the result can vary."
    }
}
